This notebook show an example on how to run predictions using a pre-trained a lgsegmentor.

This example it uses the validation data 

In [10]:
import numpy as np
import tifffile
import napari
import leopardgecko.segmentor as lgs

import logging
logging.basicConfig(level=logging.DEBUG)

# Load data  for prediction and respective labels

In [11]:
data_to_pred = tifffile.imread("test_data/TS_0005_crop_val.tif")
labels_to_compare = tifffile.imread("test_data/TS_0005_ribos_membr_crop_val.tif")

In [12]:
print(data_to_pred.shape)

(256, 256, 256)


# Setup lg segmentor

In [13]:
#Create the class
lgsegmentor0 = lgs.cMultiAxisRotationsSegmentor.create_from_model("2023-09-29_ribos_membr.lgsegmod")

DEBUG:root:cMultiAxisRotationsSegmentor __init__() with temp_data_outdir:None , cuda_device:0
DEBUG:root:load_model()


Check default settings and modify as needed

In [14]:
lgsegmentor0.NN1_pred_settings

namespace(quality='high',
          output_probs=True,
          clip_data=True,
          st_dev_factor=2.575,
          data_hdf5_path='/data',
          cuda_device=0,
          downsample=False,
          one_hot=False,
          prediction_axis='Z')

Set CUDA device.

To change cuda device don't change with .cuda_device, but rather use setcudadevice(devnumber)

You may want to check usage and memory-usage with command `!nvidia-smi` to see what is available before starting training

In [16]:
!nvidia-smi

Mon Oct 16 13:52:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                    0 |
| N/A   61C    P0   187W / 250W |  29897MiB / 32510MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   

In [17]:
lgsegmentor0.set_cuda_device(1)

Run the prediction and collect result

In [18]:
pred_labels = lgsegmentor0.predict(data_to_pred)

DEBUG:root:predict() data_in.shape:(256, 256, 256), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.376452088356018
DEBUG:root:clip_to_uint8()
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 22.28497505498702. Calculating stats.
INFO:root:Lower bound: -54.00735867823556, upper bound: 60.7602628549476
INFO:root:Number of voxels above upper bound to be clipped 95283 - percentage 0.568%
INFO:root:Number of voxels below lower bound to be clipped 175185 - percentage 1.044%
INFO:root:Rescaling intensities.
INFO:root:Data is already in integer dtype, converting to float for rescaling.
INFO:root:Converting to 

[########################################] | 100% Completed | 23.74 ss


INFO:root:NN2 prediction complete.


# Visualise result

In [ ]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})

# Compare against ground truth

Use leopardgecko metrics to get voxel-F1 (dice score)

In [ ]:
import leopardgecko.metrics as lgm

In [ ]:
lgm.MetricScoreOfVols_Dice(pred_labels,labels_to_compare)

# Run the prediction in blocks and compare with previous result

In [ ]:
import leopardgecko.utils as lgu

In [ ]:
pred_labels_block = lgu.map_vol_function_by_blocking(lgsegmentor0.predict, data_to_pred, (120,120,120),(20,32,10))

Compare predictions with and without blocking

In [ ]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})
NV.add_labels(pred_labels_block, color={1:'red'})

Prediction with blocking algorithm is worse from the calculation without blocking so be careful when using it. Thre are other instances where it may work better

Compare "block"-prediction with ground truth

In [ ]:
lgm.MetricScoreOfVols_Dice(pred_labels_block,labels_to_compare)

Compare "block"-prediction with whole prediction

In [ ]:
lgm.MetricScoreOfVols_Dice(pred_labels_block,pred_labels)